In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton
import torch
torch.__version__
# 安装 ollama
!curl -fsSL https://ollama.com/install.sh | sh

# 初始化模型

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
#     "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
#     "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
#     "unsloth/llama-3-8b-Instruct-bnb-4bit",
#     "unsloth/llama-3-70b-bnb-4bit",
#     "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
#     "unsloth/Phi-3-medium-4k-instruct",
#     "unsloth/mistral-7b-bnb-4bit",
#     "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
# ] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 SUPER. Max memory: 11.994 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


# 模型参数设置

In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    #target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],

    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# 加载数据集

In [3]:
from datasets import load_dataset, concatenate_datasets,Dataset
import pandas as pd

# 加载本地数据集
#dataset = load_dataset('json', data_files='/home/jovyan/train.jsonl', split='train')
# dataset = load_dataset("vicgalle/alpaca-gpt4", split = "train")
dataset1 = load_dataset('json', data_files='/home/jovyan/common_en_70k.jsonl', split='train')
dataset2 = load_dataset('json', data_files='/home/jovyan/common_zh_70k.jsonl', split='train')
dataset3 = load_dataset('json', data_files='/home/jovyan/computer_cn_26k_continue.jsonl', split='train')
dataset4 = load_dataset('json', data_files='/home/jovyan/computer_en_26k_continue.jsonl', split='train')
dataset6 = load_dataset('json', data_files='/home/jovyan/computer_en_26k(fixed).jsonl', split='train')
dataset5 = load_dataset('json', data_files='/home/jovyan/computer_zh_26k(fixed).jsonl', split='train')
dataset7 = load_dataset('json', data_files='/home/jovyan/unknow_zh_38k.jsonl', split='train')
dataset8 = load_dataset('json', data_files='/home/jovyan/unknow_zh_38k_continue.jsonl', split='train')
# 定义转换函数，将每个 conversation 转换为指定格式
combined_dataset = concatenate_datasets([dataset1, dataset2, dataset3,dataset4, dataset5,dataset6,dataset7, dataset8])

# 定义转换函数，将每个 conversation 转换为指定格式
def convert_conversation_format(conversation):
    new_format = {'conversations': []}

    # 遍历每个对话，将 human 和 assistant 成对地插入
    for convo in conversation:
        new_format['conversations'].append({'content': convo['human'], 'role': 'user'})
        new_format['conversations'].append({'content': convo['assistant'], 'role': 'assistant'})

    return new_format
# 遍历 dataset 并进行转换
converted_data = [convert_conversation_format(item['conversation']) for item in combined_dataset]

# 打印转换后的第一个结果

# Convert the list of converted data into a DataFrame
df = pd.DataFrame(converted_data)

# Create a Dataset from the DataFrame
converted_data_dataset = Dataset.from_pandas(df)

# Verify the dataset
print(converted_data_dataset[0])

#print(dataset[0]['conversation'])

{'conversations': [{'content': 'How much should I charge for web design by hour with profit if I need to make $8000 a month', 'role': 'user'}, {'content': "The hourly rate for web design can vary depending on your location, experience, and the type of projects you take on. However, if you need to make $8,000 a month, you would need to charge at least $160 per hour to reach that goal (assuming you work a standard 40-hour workweek). This rate would need to be adjusted based on your costs, expenses, and desired profit margin. It's important to keep in mind that this calculation is based on a standard 40 hour workweek, if you have another working schedule it may not apply.", 'role': 'assistant'}, {'content': 'What about if I spend 20hrs a week per project?', 'role': 'user'}, {'content': "If you spend 20 hours per week on a project, and you want to make $8,000 per month, you would need to charge a higher hourly rate to compensate for the fewer number of hours you're working.\nTo calculate t

# 对数据进行整合处理 并做标准化

In [4]:
### 加载数据集
from datasets import load_dataset,concatenate_datasets
# 加载本地数据集
dataset = load_dataset('json', data_files='/home/jovyan/train.jsonl', split='train')
## 数据集整合
from unsloth import to_sharegpt
dataset = to_sharegpt(
    dataset,
    merged_prompt = "{instruction}[[\nYour input is:\n{input}]]",
    output_column_name = "output",
    conversation_extension = 3, # Select more to handle longer conversations
)
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = concatenate_datasets([dataset,converted_data_dataset])

# 创建对话模板,整合到dataset中

In [5]:
chat_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{SYSTEM}<|eot_id|><|start_header_id|>user<|end_header_id|>

{INPUT}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{OUTPUT}<|eot_id|>"""
from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    default_system_message = "你是中企动力的客服",
)
print(dataset[0])

Map:   0%|          | 0/237866 [00:00<?, ? examples/s]

{'conversations': [{'content': '中企动力和中企\n中企和中企动力是一家公司吗？', 'role': 'user'}, {'content': '‘中企动力”简称‘中企”，是一家成立于1999年的老牌互联网公司', 'role': 'assistant'}, {'content': '中企可以为客户制作以下门户网站\n中企能做什么类型的门户网站？', 'role': 'user'}, {'content': '内贸营销，全球营销，通用版本', 'role': 'assistant'}, {'content': '中企动力移动商城的定义\n移动商城能做什么？', 'role': 'user'}, {'content': '助力企业打造官方独立商城\n1、轻松实现私域锁客、老客复购、熟客裂变的全方位私域运营新生态;\n2、覆盖小程序+H5两端，助力企业开展私域营销，打造忠诚客户池，创造持续销量;', 'role': 'assistant'}], 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n你是中企动力的客服<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n中企动力和中企\n中企和中企动力是一家公司吗？<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n‘中企动力”简称‘中企”，是一家成立于1999年的老牌互联网公司<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n中企可以为客户制作以下门户网站\n中企能做什么类型的门户网站？<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n内贸营销，全球营销，通用版本<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n中企动力移动商城的定义\n移动商城能做什么？<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n助力企业打造官方独立商城\n1、

# 设置训练参数

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        # num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)



# 查看资源使用
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

# 训练开始

In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 31 | Num Epochs = 15
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 13,631,488


Step,Training Loss


In [8]:
## 查看资源使用和时间

used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

185.4037 seconds used for training.
3.09 minutes used for training.
Peak reserved memory = 7.68 GB.
Peak reserved memory for training = 1.801 GB.
Peak reserved memory % of max memory = 64.032 %.
Peak reserved memory for training % of max memory = 15.016 %.


# 模型测试

In [9]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                         # Change below!
    {"role": "user",      "content": "你好呀?"}]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


我是中企动力的智能客服，有什么可以帮到你的吗？<|eot_id|>


#  保存模型,保存成 gguf格式

In [10]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving
#model.save_pretrained_gguf("model", tokenizer, quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],)

# Save to 8bit Q8_0
if True: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
# if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")
# 
# # Save to 16bit GGUF
# if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
# if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")
# 
# # Save to q4_k_m GGUF
# if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
# if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
#if False:
#    model.push_to_hub_gguf(
#        "hf/model", # Change hf to your username!
#        tokenizer,
#        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
#        token = "", # Get a token at https://huggingface.co/settings/tokens
#    )

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.58 out of 15.53 RAM for saving.


100%|███████████████████████████████████████████████████████████████████████████████████| 32/32 [00:37<00:00,  1.16s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be ./model/unsloth.Q8_0.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> Q8_0, shape = {4096, 128256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.bfloat16

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: Conversion completed! Output location: ./model/unsloth.Q8_0.gguf
Unsloth: Saved Ollama Modelfile to model/Modelfile


# 通过ollama 进行文件导入导出

In [39]:
import subprocess
subprocess.Popen(["ollama", "serve"])
import time
time.sleep(3) # Wait for a few seconds for Ollama to load!
print(tokenizer._ollama_modelfile)


2024/09/12 08:07:11 routes.go:1125: INFO server config env="map[CUDA_VISIBLE_DEVICES:0 GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/home/jovyan/.ollama/models OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://*] OLLAMA_RUNNERS_DIR: OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_DEVICES:]"
time=2024-09-12T08:07:11.841Z level=INFO source=images.go:753 msg="total blobs: 37"
time=2024-09-12T08:07:11.961Z level=INFO source=images.go:760 msg="

FROM {__FILE_LOCATION__}

TEMPLATE """{{ if .System }}<|start_header_id|>system<|end_header_id|>

{{ .System }}<|eot_id|>{{ end }}{{ if .Prompt }}<|start_header_id|>user<|end_header_id|>

{{ .Prompt }}<|eot_id|>{{ end }}<|start_header_id|>assistant<|end_header_id|>

{{ .Response }}<|eot_id|>"""

PARAMETER stop "<|python_tag|>"
PARAMETER stop "<|end_header_id|>"
PARAMETER stop "<|end_of_text|>"
PARAMETER stop "<|eom_id|>"
PARAMETER stop "<|start_header_id|>"
PARAMETER stop "<|finetune_right_pad_id|>"
PARAMETER stop "<|eot_id|>"
PARAMETER stop "<|reserved_special_token_"
SYSTEM "你是中企动力的客服"


In [43]:
!cp /home/jovyan/Modelfile ./model/Modelfile

!cat ./model/Modelfile

FROM ./model/unsloth.Q8_0.gguf
TEMPLATE """{{- if or .System .Tools }}<|start_header_id|>system<|end_header_id|>
{{- if .System }}

{{ .System }}
{{- end }}
{{- if .Tools }}

Cutting Knowledge Date: December 2023

When you receive a tool call response, use the output to format an answer to the orginal user question.

You are a helpful assistant with tool calling capabilities.
{{- end }}
{{- end }}<|eot_id|>
{{- range $i, $_ := .Messages }}
{{- $last := eq (len (slice $.Messages $i)) 1 }}
{{- if eq .Role "user" }}<|start_header_id|>user<|end_header_id|>
{{- if and $.Tools $last }}

Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.

Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}. Do not use variables.

{{ range $.Tools }}
{{- . }}
{{ end }}
{{- end }}

{{ .Content }}<|eot_id|>{{ if $last }}<|start_header_id|>assistant<|end_header_id|>

{{ end }

In [44]:
!ollama create wt0919 -f ./model/Modelfile
!kill -9 $(pgrep -f ollama)
print("结束")

[GIN] 2024/09/12 - 08:08:58 | 200 |    1.056486ms |       127.0.0.1 | HEAD     "/"
transferring model data ⠋ transferring model data ⠙ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transferring model data ⠴ transferring model data ⠦ transferring model data ⠧ transferring model data ⠇ transferring model data ⠏ transferring model data ⠋ transferring model data ⠙ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transferring model data ⠴ transferring model data ⠦ transferring model data ⠧ transferring model data ⠇ transferring model data ⠏ transferring model data ⠋ transferring model data ⠙ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transferring model data ⠴ transferring model data ⠦ transferring model data ⠧ transferring model data ⠇ transferring model data ⠏ transferring model data ⠋ transferring model data ⠙ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transfe